In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pickle
import os

In [2]:
%matplotlib inline

In [3]:
# Create folder to store csv files:
dir_name = './DATA'
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

# Creating Database

## Vehicle Data

In [4]:
years = [2016, 2017, 2018, 2019, 2020]

In [5]:
def clean_state(x):
    if type(x) != str:
        print(x)
    for char in '(1234567890)':
        x = x.replace(char,'')
    x = x.strip(' ')
    return x

# Read in dataframe
vehicle_dfs = {}
for year in years:
    url = "https://www.fhwa.dot.gov/policyinformation/statistics/{}/mv1.cfm".format(year)
    html = requests.get(url).content
    tempdf = pd.read_html(html)[0]

    # Select correct columns to use
    tempdf.columns = list(range(16))
    tempdf = tempdf[[0,3,6,9,12]]
    # Rename columns
    tempdf.columns = ['State', 'Automobiles', 'Buses', 'Trucks', 'Motorcycles']
    # add 'Year' Column
    tempdf['Year'] = year
    # Reorder Column
    tempdf = tempdf[['State', 'Year', 'Automobiles', 'Buses', 'Trucks', 'Motorcycles']]
    # Drop Columns with NaNs
    tempdf.dropna(inplace=True)
    # Drop 'Total' Row:
    tempdf = tempdf[tempdf['State'] != 'Total']
    # Clean 'State' Column
    tempdf['State'] = tempdf['State'].map(clean_state)
    vehicle_dfs[year] = tempdf

In [6]:
# Combine dfs into one
dfs = [df for df in vehicle_dfs.values()]
vehicle_df = pd.concat(dfs, axis=0, ignore_index=True)
vehicle_df[['Automobiles', 'Buses', 'Trucks', 'Motorcycles']] = vehicle_df[['Automobiles', 'Buses', 'Trucks', 'Motorcycles']].astype(int)
vehicle_df

,State,Year,Automobiles,Buses,Trucks,Motorcycles
0,Alabama,2016,2284443,6238,3067917,109703
1,Alaska,2016,183259,8119,571287,31949
2,Arizona,2016,2377962,9016,3233330,166583
3,Arkansas,2016,942604,11931,1762765,90838
4,California,2016,14768392,98622,14511913,842106
...,...,...,...,...,...,...
250,Virginia,2020,3057254,35463,4329294,184441
251,Washington,2020,2800370,24172,4211411,221448
252,West Virginia,2020,508663,3155,1102015,43529
253,Wisconsin,2020,1901497,14941,3423523,276310


In [7]:
vehicle_df.to_csv(os.path.join(dir_name, 'vehicle_data.csv'), index=False)

## Population data

Before running this, download [*popest-annual.xls*](https://www.icip.iastate.edu/tables/population/states-estimates) and place into **./DATA**

Also, install xlrd

In [8]:
years = list(range(2010, 2020))

In [9]:
# Install xlrd to run this!
pop_df = pd.read_excel(os.path.join(dir_name, 'popest-annual.xls'), sheet_name='States', skiprows=[0,1,2,3,4,5,7,8,9,10,11])
pop_df.drop(columns = ['Fips', 'Estimates Base (4/1/2010)'], inplace=True)

In [10]:
pop_df.head()

,Area,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,4785514.0,4799642.0,4816632.0,4831586.0,4843737.0,4854803.0,4866824.0,4877989.0,4891628.0,4907965.0
1,Alaska,713982.0,722349.0,730810.0,737626.0,737075.0,738430.0,742575.0,740983.0,736624.0,733603.0
2,Arizona,6407342.0,6473416.0,6556344.0,6634690.0,6732873.0,6832810.0,6944767.0,7048088.0,7164228.0,7291843.0
3,Arkansas,2921998.0,2941038.0,2952876.0,2960459.0,2968759.0,2979732.0,2991815.0,3003855.0,3012161.0,3020985.0
4,California,37319550.0,37636311.0,37944551.0,38253768.0,38586706.0,38904296.0,39149186.0,39337785.0,39437463.0,39437610.0


In [11]:
year_dfs = {}
for year in years:
    new_df = pop_df[['Area', year]]
    new_df = new_df.rename(columns={'Area': 'State', year:'Population'})
    # Remove all not-state rows:
    new_df = new_df.iloc[:51]
    new_df['Population'] = new_df['Population'].astype(int)
    new_df['Year'] = year
    new_df = new_df[['State', 'Year', 'Population']]

    year_dfs[year] = new_df

In [12]:
dfs = [df for df in year_dfs.values()]
pop_df = pd.concat(dfs, axis=0, ignore_index=True)
pop_df

,State,Year,Population
0,Alabama,2010,4785514
1,Alaska,2010,713982
2,Arizona,2010,6407342
3,Arkansas,2010,2921998
4,California,2010,37319550
...,...,...,...
505,Virginia,2019,8556642
506,Washington,2019,7614024
507,West Virginia,2019,1795263
508,Wisconsin,2019,5824581


In [13]:
pop_df.to_csv(os.path.join(dir_name, 'pop_data.csv'), index=False)

## Drivers License Minimum Age

In [14]:
url = "https://www.verywellfamily.com/driving-age-by-state-2611172#toc-learners-permits"
html = requests.get(url).content
minage_df = pd.read_html(html)[0]
minage_df.columns = minage_df.iloc[0]
minage_df.drop(index=[0], inplace=True)
minage_df.set_index('State', inplace=True)
minage_df.rename(columns = {"Learner's Permit":'Learners_Permit', "Restricted License":"Restricted_License", "Full License":"Full_License"}, inplace=True)
minage_df.head()

,Learners_Permit,Restricted_License,Full_License
State,,,
Alabama,15,16,17
Alaska,14,16,"16, 6 mos."
Arizona,"15, 6 mos.",16,"16, 6 mos."
Arkansas,14,16,18
California,"15, 6 mos.",16,17


In [15]:
def clean_age(val):
    if type(val) == float:
        return val
    age_split = val.split(', ')
    age = float(age_split[0])
    if len(age_split) > 1:
        age += float(age_split[1][0])/12
    return age

minage_df['Learners_Permit'] = minage_df['Learners_Permit'].map(clean_age)
minage_df['Restricted_License'] = minage_df['Restricted_License'].map(clean_age)
minage_df['Full_License'] = minage_df['Full_License'].map(clean_age)

In [16]:
minage_df.head()

,Learners_Permit,Restricted_License,Full_License
State,,,
Alabama,15.0,16.0,17.0
Alaska,14.0,16.0,16.5
Arizona,15.5,16.0,16.5
Arkansas,14.0,16.0,18.0
California,15.5,16.0,17.0


In [17]:
minage_df.to_csv(os.path.join(dir_name, 'min_age_data.csv'))

## Speed Limit data

In [18]:
url = "https://www.iihs.org/topics/speed/speed-limit-laws"
html = requests.get(url).content
limit_df = pd.read_html(html)[0]
limit_df.columns = ['State', 'Rural_Interstate', 'Urban_Interstate', 'Other_Limited_Access', 'Other']
limit_df.head()

,State,Rural_Interstate,Urban_Interstate,Other_Limited_Access,Other
0,Alabama,70,65,65,65
1,Alaska,65,55,65,55
2,Arizona,75,65,65,65 trucks: 65
3,Arkansas,75 trucks: 70,65,75 trucks: 70,65
4,California,70; trucks: 55,65 trucks: 55,70 trucks: 55,65 trucks: 55


In [19]:
def clean_speed(val):
    if type(val) == float:
        return val
    else:
        try:
            return int(str(val)[:2])
        except:
            return int(str(val)[5:7])

for col in ['Rural_Interstate', 'Urban_Interstate', 'Other_Limited_Access', 'Other']:
    limit_df[col] = limit_df[col].map(clean_speed)

In [20]:
limit_df.head()

,State,Rural_Interstate,Urban_Interstate,Other_Limited_Access,Other
0,Alabama,70.0,65,65.0,65
1,Alaska,65.0,55,65.0,55
2,Arizona,75.0,65,65.0,65
3,Arkansas,75.0,65,75.0,65
4,California,70.0,65,70.0,65


In [21]:
limit_df.to_csv(os.path.join(dir_name, 'speedlimit_data.csv'), index=False)

## Licensed Drivers

Download csv from [here](https://datahub.transportation.gov/Roadways-and-Bridges/Licensed-Drivers-by-state-gender-and-age-group/xfkb-3bxx)  
Place it in **./DATA**, renamed to *licensed_drivers.csv*


In [22]:
license_df = pd.read_csv(os.path.join(dir_name, 'licensed_drivers.csv'))
license_df.rename(columns={'Cohort':'Age'}, inplace=True)
# Reorder columns
license_df = license_df[['State', 'Year', 'Gender', 'Age', 'Drivers']]
# Only keep 2010 and above
license_df = license_df[license_df['Year'] >= 2010]
# Cleaning
license_df.fillna(0, inplace=True)
license_df['Drivers'] = license_df['Drivers'].astype(int)
license_df.replace({'Under 16':'15-'}, inplace=True)
license_df.head()

,State,Year,Gender,Age,Drivers
0,Alabama,2017,Male,15-,0
1,Alaska,2017,Male,15-,0
2,Arizona,2017,Male,15-,0
3,Arkansas,2017,Male,15-,0
4,California,2017,Male,15-,0


In [23]:
early_20s = license_df[license_df['Age'].isin(['19', '20', '21', '22', '23', '24'])]
early_20s = early_20s.groupby(['State', 'Year', 'Gender'], as_index=False).sum(numeric_only=True)
early_20s['Age'] = '19-24'
early_20s = early_20s[['State', 'Year', 'Gender', 'Age', 'Drivers']]

In [24]:
license_df = license_df[~license_df['Age'].isin(['19', '20', '21', '22', '23', '24'])]

In [25]:
license_df = pd.concat([license_df, early_20s])

In [26]:
# Sort
license_df.sort_values(['Year', 'State', 'Gender', 'Age'], inplace=True)

In [27]:
license_df.to_csv(os.path.join(dir_name, 'license_data.csv'), index=False)

## Temperature Data

In [37]:
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

In [38]:
all = {}
for month in months:
    url = "https://www.extremeweatherwatch.com/us-state-averages/month-{}".format(month)
    html = requests.get(url).content
    tempdf = pd.read_html(html)[0]
    tempdf.columns = ['State', 'High_F', 'Low_F', 'Precipitation_in']
    tempdf.set_index('State', inplace=True)
    tempdf['Month'] = month[0].upper() + month[1:]
    tempdf['Avg_F'] = (tempdf['High_F'] + tempdf['Low_F'])/2
    tempdf= tempdf[['Month', 'Low_F', 'Avg_F', 'High_F', 'Precipitation_in']]
    all[month] = tempdf
    
temperature_df = pd.concat(all.values(), axis=0)

In [40]:
temperature_df.to_csv(os.path.join(dir_name, 'temperature_data.csv'), index=True)